In [12]:
#!pip install flask
#!pip install -U openai-whisper
#!pip install pytorch
!pip install ffmpeg 

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6089 sha256=c294b79a3d37b062cf790243e0cb233fb93a3e3c79c48744f74c9d5b3bf0e32a
  Stored in directory: c:\users\ssafy\appdata\local\pip\cache\wheels\26\21\0c\c26e09dff860a9071683e279445262346e008a9a1d2142c4ad
Successfully built ffmpeg


In [1]:
import os  # 운영 체제와 상호작용하기 위한 os 모듈 임포트
import tempfile  # 임시 파일 생성을 위한 tempfile 모듈 임포트
import logging  # 로그 메시지를 위한 logging 모듈 임포트
from flask import Flask, request, jsonify  # 웹 애플리케이션 생성을 위한 Flask와 유틸리티 임포트
import whisper  # Whisper ASR 모델 임포트
import torch  # 텐서 연산을 위한 PyTorch 임포트
import threading  # 스레드를 생성하기 위한 threading 모듈 임포트
import queue  # 작업 큐 생성을 위한 queue 모듈 임포트
import uuid  # 고유한 작업 ID 생성을 위한 uuid 모듈 임포트

# 로그 메시지를 info 레벨로 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # 이 모듈을 위한 로거 생성

app = Flask(__name__)  # Flask 애플리케이션 인스턴스 생성

# 작업 큐와 결과 저장을 위한 딕셔너리 초기화
task_queue = queue.Queue()
results = {}

# 오디오 파일을 처리할 작업자 스레드 수 설정
num_worker_threads = 1

# 오디오 파일을 처리할 작업자 함수 정의
def worker(thread_id):
    logger.info(f"Worker {thread_id} starting, loading Whisper model...")
    whisper_model = whisper.load_model("medium")  # Whisper ASR 모델 로드
    logger.info(f"Worker {thread_id} loaded Whisper model.")

    while True:  # 작업자가 계속 실행되도록 무한 루프
        task_id, audio_path = task_queue.get()  # 큐에서 작업 가져오기
        if task_id is None:  # 종료 신호 확인
            break  # 종료 신호를 받으면 루프 종료
        
        logger.info(f"Worker {thread_id} processing task {task_id}")
        
        try:
            # Whisper 모델을 사용하여 오디오 파일을 전사
            results[task_id] = {"status": "processing"}
            result = whisper_model.transcribe(audio_path)
            # segment에서 id, start, end, text 
            # 비동기 
            # API통신.uri(result, "spring url")
            
            
            results[task_id] = {"status": "completed", "result": result}  # 결과를 딕셔너리에 저장
        except Exception as e:
            logger.error(f"Worker {thread_id} encountered an error: {str(e)}")  # 발생한 오류 로그
            results[task_id] = {"status": "failed", "error": str(e)}  # 오류를 결과에 저장
        finally:
            os.remove(audio_path)  # 임시 오디오 파일 제거
            if torch.cuda.is_available():  # CUDA 사용 가능 여부 확인
                torch.cuda.empty_cache()  # 필요 시 CUDA 메모리 캐시 지우기
        
        task_queue.task_done()  # 작업 완료 표시

# 작업자 스레드 시작
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker, args=(i,))  # 작업자 함수에 대한 새 스레드 생성
    t.start()  # 스레드 시작
    threads.append(t)  # 스레드 목록에 추가

@app.route('/', methods=['GET'])  # 홈 페이지를 위한 라우트 정의
def home():
    return "Welcome to the Asynchronous Flask Server with Whisper!"  # 환영 메시지 반환

@app.route('/stt', methods=['POST'])  # STT 요청을 위한 라우트 정의
def stt_request():
    logger.info("Received STT request")
    
    if 'audio' not in request.files:  # 오디오 파일이 제공되었는지 확인
        logger.warning("No audio file provided in the request")
        return jsonify({'error': 'No audio file provided'}), 400  # 제공되지 않으면 오류 반환
    
    audio_data = request.files['audio']  # 요청에서 오디오 파일 가져오기
    
    if audio_data.filename == '':  # 파일 이름이 비어 있는지 확인
        logger.warning("Empty filename provided")
        return jsonify({'error': 'No selected file'}), 400  # 비어 있으면 오류 반환
    
    logger.info(f"Processing audio file: {audio_data.filename}")
    
    # 오디오 데이터를 저장할 임시 파일 생성
    with tempfile.NamedTemporaryFile(delete=False) as temp_audio:
        audio_data.save(temp_audio.name)  # 오디오 데이터를 임시 파일에 저장
        temp_audio_path = temp_audio.name  # 임시 파일 경로 저장
    
    # 고유한 작업 ID를 생성하고 작업을 큐에 추가
    task_id = str(audio_data.filename)  # 작업을 위한 고유 식별자 생성
    task_queue.put((task_id, temp_audio_path))  # 큐에 작업 추가
    
    logger.info(f"Task {task_id} added to the queue")
    
    return jsonify({"task_id": task_id, "status": "processing"}), 202  # 작업 ID와 상태 반환

@app.route('/stt_result/<task_id>', methods=['GET'])  # 작업 상태를 확인하기 위한 라우트 정의
def get_status(task_id):
    if task_id not in results:  # 작업 ID가 결과에 없으면
        return jsonify({"status": "No_result"}), 404  # 처리 중 상태 반환
    
    result = results[task_id]  # 주어진 작업 ID에 대한 결과 가져오기
    
    if result["status"] == "completed":  # 작업이 완료되었는지 확인
        del results[task_id]  # 메모리 해제를 위해 결과 딕셔너리에서 제거
        return jsonify(result), 200  # 완료된 결과 반환
    elif result["status"] == "failed":  # 작업이 실패했는지 확인
        del results[task_id]  # 메모리 해제를 위해 결과 딕셔너리에서 제거
        return jsonify(result), 500  # 실패한 결과 반환
    else:
        return jsonify({"status": "processing"}), 202  # 여전히 처리 중인 상태 반환

@app.errorhandler(500)  # 500 내부 서버 오류 처리
def internal_error(error):
    logger.error(f"Internal Server Error: {str(error)}")  # 오류 로그
    return jsonify({"error": "Internal Server Error"}), 500  # 오류 메시지 반환

if __name__ == '__main__':
    try:
        app.run(host='0.0.0.0', port=5000)  # 포트 5000에서 Flask 앱 시작
    finally:
        # 애플리케이션 종료 시 작업자 스레드 중지
        torch.cuda.empty_cache()  # 메모리 캐시 비우기
        for _ in range(num_worker_threads):
            task_queue.put((None, None))  # 스레드에 중지 신호 전송
        for t in threads:
            t.join()  # 모든 스레드가 종료될 때까지 대기


INFO:__main__:Worker 0 starting, loading Whisper model...


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://70.12.246.80:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Received STT request
INFO:werkzeug:127.0.0.1 - - [27/Sep/2024 10:44:14] "POST /stt HTTP/1.1" 400 -
C:\Users\SSAFY\AppData\Local\Programs\Python\Python312\Lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary

In [3]:
import os  # 운영 체제와 상호작용하기 위한 os 모듈 임포트
import tempfile  # 임시 파일 생성을 위한 tempfile 모듈 임포트
import logging  # 로그 메시지를 위한 logging 모듈 임포트
from flask import Flask, request, jsonify  # 웹 애플리케이션 생성을 위한 Flask와 유틸리티 임포트
import whisper  # Whisper ASR 모델 임포트
import torch  # 텐서 연산을 위한 PyTorch 임포트
import threading  # 스레드를 생성하기 위한 threading 모듈 임포트
import queue  # 작업 큐 생성을 위한 queue 모듈 임포트
import uuid  # 고유한 작업 ID 생성을 위한 uuid 모듈 임포트
import requests

# 로그 메시지를 info 레벨로 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)  # 이 모듈을 위한 로거 생성

app = Flask(__name__)  # Flask 애플리케이션 인스턴스 생성

# 작업 큐와 결과 저장을 위한 딕셔너리 초기화
task_queue = queue.Queue()
results = {}

# 오디오 파일을 처리할 작업자 스레드 수 설정
num_worker_threads = 1

# 오디오 파일을 처리할 작업자 함수 정의
def worker(thread_id):
    logger.info(f"Worker {thread_id} starting, loading Whisper model...")
    model = whisper.load_model("base")  # Whisper ASR 모델 로드
    logger.info(f"Worker {thread_id} loaded Whisper model.")

    while True:  # 작업자가 계속 실행되도록 무한 루프
        note_id, audio_path = task_queue.get()  # 큐에서 작업 가져오기
        if note_id is None:  # 종료 신호 확인
            break  # 종료 신호를 받으면 루프 종료
        
        logger.info(f"Worker {thread_id} processing task {note_id}")
        
        try:
            # Whisper 모델을 사용하여 오디오 파일을 전사
            # results[note_id] = {"status": "processing"}
            result = model.transcribe(audio_path)
            
            send_result_to_spring(note_id, result['segments'])
            
            # results[note_id] = {"status": "completed", "result": result}  # 결과를 딕셔너리에 저장
               
        except Exception as e:
            logger.error(f"Worker {thread_id} encountered an error: {str(e)}")  # 발생한 오류 로그
            results[note_id] = {"status": "failed", "error": str(e)}  # 오류를 결과에 저장
        
        finally:
            os.remove(audio_path)  # 임시 오디오 파일 제거
            if torch.cuda.is_available():  # CUDA 사용 가능 여부 확인
                torch.cuda.empty_cache()  # 필요 시 CUDA 메모리 캐시 지우기
        
        task_queue.task_done()  # 작업 완료 표시

def send_result_to_spring(note_id, result):
    # 결과를 Spring 서버에 비동기로 전송
    data = {
        "id": note_id,  # note_id를 문자열로 변환하지 않음
        "result": result  # result는 JSON 형식으로 예상
    }
    
    try:
        response = requests.post("http://localhost:8080/voice/stt", json=data)
        response.raise_for_status()  # 잘못된 응답에 대한 에러 발생
    except requests.exceptions.RequestException as e:
        print(f"결과 전송 오류: {e}")


# 작업자 스레드 시작
threads = []
for i in range(num_worker_threads):
    t = threading.Thread(target=worker, args=(i,))  # 작업자 함수에 대한 새 스레드 생성
    t.start()  # 스레드 시작
    threads.append(t)  # 스레드 목록에 추가

@app.route('/', methods=['GET'])  # 홈 페이지를 위한 라우트 정의
def home():
    return "Welcome to the Asynchronous Flask Server with Whisper!"  # 환영 메시지 반환

@app.route('/stt', methods=['POST'])  # STT 요청을 위한 라우트 정의
def stt_request():
    logger.info("Received STT request")
    
    if not request.json.get('presigned_url'):  # 오디오 파일이 제공되었는지 확인
        logger.warning("No audio file provided in the request")
        return jsonify({'error': 'No audio file provided'}), 400  # 제공되지 않으면 오류 반환

    # https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/codeapple.mp3
    audio_uri = request.json.get('presigned_url')  # 요청에서 오디오 파일 가져오기
    note_id = request.json.get('id')
    
    if audio_uri:  # audio_uri가 제공된 경우
        try:
            logger.info(f"Downloading audio file from URI: {audio_uri}")
            response = requests.get(audio_uri)  # URI에서 파일 다운로드
            response.raise_for_status()  # 오류 발생 시 예외 처리
            
            # 오디오 데이터를 저장할 임시 파일 생성
            with tempfile.NamedTemporaryFile(delete=False) as temp_audio:
                temp_audio.write(response.content)  # 다운로드한 파일 내용을 임시 파일에 저장
                temp_audio_path = temp_audio.name  # 임시 파일 경로 저장
            
            logger.info(f"Audio file downloaded and saved to {temp_audio_path}")

            task_queue.put((note_id, temp_audio_path))  # 큐에 작업 추가

            logger.info(f"Task {note_id} added to the queue")
    
            return jsonify({"note_id": note_id, "status": "processing"}), 202  # 작업 ID와 상태 반환
            
        except requests.exceptions.RequestException as e:
            logger.error(f"Failed to download audio file: {str(e)}")
            return jsonify({'error': 'Failed to download audio file'}), 400          

@app.errorhandler(500)  # 500 내부 서버 오류 처리
def internal_error(error):
    logger.error(f"Internal Server Error: {str(error)}")  # 오류 로그
    return jsonify({"error": "Internal Server Error"}), 500  # 오류 메시지 반환

if __name__ == '__main__':
    try:
        app.run(host='localhost', port=5000)  # 포트 5000에서 Flask 앱 시작
    finally:
        # 애플리케이션 종료 시 작업자 스레드 중지
        torch.cuda.empty_cache()  # 메모리 캐시 비우기
        for _ in range(num_worker_threads):
            task_queue.put((None, None))  # 스레드에 중지 신호 전송
        for t in threads:
            t.join()  # 모든 스레드가 종료될 때까지 대기


INFO:__main__:Worker 0 starting, loading Whisper model...


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Worker 0 loaded Whisper model.
INFO:__main__:Received STT request
INFO:__main__:Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/codeapple.mp3
INFO:__main__:Audio file downloaded and saved to C:\Users\SSAFY\AppData\Local\Temp\tmpp2oe6hlm
INFO:__main__:Task 1 added to the queue
INFO:__main__:Worker 0 processing task 1
INFO:werkzeug:127.0.0.1 - - [27/Sep/2024 12:45:27] "POST /stt HTTP/1.1" 202 -
INFO:__main__:Received STT request
INFO:__main__:Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/codeapple.mp3
INFO:__main__:Audio file downloaded and saved to C:\Users\SSAFY\AppData\Local\Temp\tmphwhp2eps
INFO:__main__:Task 1 added to the queue
INFO:__main__:Worker 0 processing task 1
INFO:werkzeug:127.0

결과 전송 오류: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /voice/stt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000200E1435370>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


INFO:__main__:Received STT request
INFO:__main__:Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/codeapple.mp3
INFO:__main__:Audio file downloaded and saved to C:\Users\SSAFY\AppData\Local\Temp\tmpsv206id8
INFO:__main__:Task 1 added to the queue
INFO:__main__:Worker 0 processing task 1
INFO:werkzeug:127.0.0.1 - - [27/Sep/2024 13:08:12] "POST /stt HTTP/1.1" 202 -
INFO:__main__:Received STT request
INFO:__main__:Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/codeapple.mp3
INFO:__main__:Audio file downloaded and saved to C:\Users\SSAFY\AppData\Local\Temp\tmpet8n92e3
INFO:__main__:Task 1 added to the queue
INFO:werkzeug:127.0.0.1 - - [27/Sep/2024 13:08:17] "POST /stt HTTP/1.1" 202 -
INFO:__main__:Received STT request
INFO:__main__:Downloading audio file from URI: https://timeisnullnull.s3.ap-northeast-2.amazonaws.com/codeapple.mp3
INFO:__main__:Audio file downloaded and saved to C:\Users\SSAFY\AppData\Local\Te

...

결과 전송 오류: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /voice/stt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000200E1436C30>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


INFO:__main__:Worker 0 processing task 1


결과 전송 오류: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /voice/stt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002014AB563C0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


INFO:__main__:Worker 0 processing task 1


결과 전송 오류: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /voice/stt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002012403C1D0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))
결과 전송 오류: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /voice/stt (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000200B95F7FE0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))


In [2]:
import torch

torch.cuda.empty_cache()

In [7]:
import ffmpeg

# ffmpeg 실행 결과를 가져오기
version = ffmpeg.probe('version')

print(version)


AttributeError: module 'ffmpeg' has no attribute 'probe'